In [1]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

# Assuming the functions are HTTP endpoints
function1_url = 'https://europe-west3-lumpito.cloudfunctions.net/download_files'
function2_url = 'https://europe-west3-lumpito.cloudfunctions.net/extract_table'
function3_url = 'https://europe-west3-lumpito.cloudfunctions.net/standardize_data'
#numbers 9 to 100 in the company_ids list

# company_ids =[23,30,124,153,190]
#array with numbers from 1 to 200
company_ids = list(range(201, 10930))

import time

# Constants for retry policy
MAX_RETRIES = 3
BACKOFF_FACTOR = 1.5

def call_function(url, json, session):
    for attempt in range(MAX_RETRIES):
        response = session.post(url, json=json)
        if response.status_code == 200:
            return response.json()  # Assuming the function returns JSON
        else:
            time.sleep(BACKOFF_FACTOR ** attempt)  # Exponential backoff
    return None  # All retries failed

def call_function_in_sequence(company_id):
    session = requests.Session()  # Reuse connections within the function
    json_data = {"company_id": company_id}
    json_one = {"company_id": company_id, "documents": "si"}
    #create a dataframe with the company_id and the result of the function
    df_function_success = pd.DataFrame(columns=['company_id', 'function1', 'function2', 'function3'])

    # # First function call with retry
    result1 = call_function(function1_url, json_one, session)
    if not result1:
        #add the company_id and the result of the function to the dataframe
        df_function_success = df_function_success.concat({'company_id': company_id, 'function1':"failed"})
        return company_id, False, 'Function 1 failed'

    #Second function call with retry
    result2 = call_function(function2_url, json_data, session)
    if not result2:
        df_function_success = df_function_success.concat({'company_id': company_id, 'function2':"failed"})
        return company_id, False, 'Function 2 failed'

    #Third function call with retry
    result3 = call_function(function3_url, json_data, session)
    if not result3:
        df_function_success = df_function_success.concat({'company_id': company_id, 'function3':"failed"})
        return company_id, False, 'Function 3 failed'

    return company_id, True, 'Success'

# The rest of your code remains the same

# The number of workers can be adjusted based on your system and network capabilities
workers = 500

with ThreadPoolExecutor(max_workers=workers) as executor:
    # Dictionary to keep track of futures for each company_id
    future_to_company_id = {executor.submit(call_function_in_sequence, cid): cid for cid in company_ids}

    for future in as_completed(future_to_company_id):
        company_id = future_to_company_id[future]
        try:
            _, success, message = future.result()
            if success:
                print(f"Company ID {company_id}: All functions succeeded")
            else:
                print(f"Company ID {company_id}: {message}")
        except Exception as exc:
            print(f"Company ID {company_id} generated an exception: {exc}")

Company ID 9: All functions succeeded
Company ID 3: All functions succeeded
Company ID 12: All functions succeeded
Company ID 1: All functions succeeded
Company ID 7: All functions succeeded
Company ID 6: Function 1 failed
Company ID 4: Function 1 failed
Company ID 8: Function 1 failed
Company ID 5: Function 1 failed
Company ID 10: Function 1 failed
Company ID 19: All functions succeeded
Company ID 11: All functions succeeded
Company ID 15: Function 1 failed
Company ID 16: All functions succeeded
Company ID 2: Function 1 failed
Company ID 13: Function 1 failed
Company ID 18: All functions succeeded
Company ID 14: Function 1 failed
Company ID 22: All functions succeeded
Company ID 24: All functions succeeded
Company ID 17: Function 1 failed
Company ID 29: All functions succeeded
Company ID 20: Function 1 failed
Company ID 26: All functions succeeded
Company ID 21: Function 1 failed
Company ID 27: All functions succeeded
Company ID 25: All functions succeeded
Company ID 23: Function 1 fa

In [4]:
import requests
session = requests.Session() 
# json={"company_id": 11, "documents": "gs"}
json={"company_id": 25}
function1_url = 'https://europe-west3-lumpito.cloudfunctions.net/extract_table'
response1 = session.post(function1_url, json=json)
# function1_url = 'https://europe-west3-lumpito.cloudfunctions.net/standardize_data'
# response1 = session.post(function1_url, json=json)
print(response1.text)

{"Vor- und Nachname (mit Geburtsdatum und Wohnort)":{"0":"Firma (mit Sitz, Register- gericht und -Nr.)","1":"Valentine Ventures UG (haftungsbeschr\u00e4nkt), Ottersberg, (AG Wals- rode, HRB 204137)","2":"Gesamt"},"Ge- sch\u00e4fts- anteil":{"0":"Nr.","1":"1","2":""},"Nenn- betrag":{"0":"(EURO)","1":"25.000 \u20ac","2":"25.000 \u20ac"},"Anteil am Stammkapital je Gesch\u00e4ftsanteil":{"0":"in Prozent","1":"100 %","2":"100 %"},"Gesamtanteil am Stammkapital je Gesellschafter in":{"0":"Prozent","1":"100 %","2":"100 %"}}


In [6]:
import requests
session = requests.Session() 
# json={"company_id": 11, "documents": "gs"}
json={"company_id": 4}
function1_url = 'https://europe-west3-lumpito.cloudfunctions.net/get_structured_content'
response1 = session.post(function1_url, json=json)
# function1_url = 'https://europe-west3-lumpito.cloudfunctions.net/standardize_data'
# response1 = session.post(function1_url, json=json)
print(response1.text)

500 Internal Server Error: The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.
